# Making an AI girlfriend based on Samantha from *Her* (2014)
This notebook is virtually the same as my first attempt, except I will be also implementing dialogue from Ava from *Ex Machina* to the dataset. 

In this notebook, I am going over my first attempt to create a GPT2 model that resembles an AI girlfriend based on lines from movies and video games. And what better character to start with than Samantha from *Her,* an actual fictional AI girlfriend.

If this goes well, I will complexify the experiment with more characters and work towards a more sophisticated initial experiment.

## Getting the Dialogue
First, we need to use Python to extract lines of dialogue from the movie script text file.

I'll use this first chunk of code to install our required libraries and then go through and make a file of just Samantha's dialogue.

After this, i will perform the same function, redefining the variables.

In [1]:
import re
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  # This disables TensorFlow 2.x behaviors and enables 1.x behaviors

import gpt_2_simple as gpt
from datetime import datetime
import requests

# Download GPT-2 model
gpt.download_gpt2(model_name="124M")

Instructions for updating:
non-resource variables are not supported in the long term


Fetching checkpoint: 1.05Mit [00:00, 3.83Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 2.62Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 5.46Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:20, 24.2Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 1.75Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 2.80Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 3.44Mit/s]                                                      


*Note that I did get this code from ChatGPT with minor modifications to help support my limited Python skills*

In [6]:
def extract_character_dialogue(script_file, character_name, output_file):
    # Open the script file and read the content
    with open(script_file, 'r', encoding='ISO-8859-1') as file:
        script_content = file.read()

    # Create a regular expression pattern to match character's dialogue
    # Assuming the character's name is followed by a newline, then the dialogue on the next line
    pattern = re.compile(r'(?<=\n)(' + re.escape(character_name) + r')\n([^\n]+)', re.IGNORECASE)

    # Find all the matches
    dialogues = pattern.findall(script_content)

    # Open the output file to save the character's dialogues
    with open(output_file, 'w', encoding='ISO-8859-1') as output:
        for character, dialogue in dialogues:
            output.write(f"{dialogue}\n")

    print(f"Dialogue for {character_name} has been extracted to {output_file}")
# defining our files and characters
script_file = 'Her.txt'  
character_name = 'SAMANTHA' 
output_file = 'sam_dialogue.txt'  

extract_character_dialogue(script_file, character_name, output_file)

Dialogue for SAMANTHA has been extracted to sam_dialogue.txt


In [7]:
# redefining our files and characters
script_file = 'exMachina.txt'  
character_name = 'AVA' 
output_file = 'ava_dialogue.txt'  

extract_character_dialogue(script_file, character_name, output_file)

Dialogue for AVA has been extracted to ava_dialogue.txt


### Concatenating our AI GF Dialogue

Lastly, I will concatenate our dialogue from the two text files into one mega file.

In [8]:
# Putting the files in a list to concatenate
file_names = [
    "sam_dialogue.txt","ava_dialogue.txt"
]

# The concatenation... 
with open("GFdata.txt", 'w', encoding='ISO-8859-1') as outfile:
    for file_name in file_names:
        with open(file_name, 'r', encoding='ISO-8859-1') as infile:
            outfile.write(infile.read())
    print("Concatenated all text files into GFdata.txt")

Concatenated all text files into GFdata.txt


# Training the Model

Next, I will train GPT2 to take in Samantha's dialogue and learn from it to use it when constructing its own responses. This is developed by code from Dr. Shawn Graham's workbook, "Practical Necromancy"

In [9]:
# Clear any previous session (important to prevent variable reuse issues)
tf.reset_default_graph()  # Clear the TensorFlow graph

# Start TensorFlow session for GPT-2
sess = gpt.start_tf_sess()

# Fine-tune the model
gpt.finetune(sess,
              dataset="GFdata.txt",
              model_name='124M',
              steps=100,  # You might want to increase this for better results
              restore_from='fresh',
              run_name='run1',
              print_every=10,  # Print frequency
              sample_every=20,  # Sample frequency
              save_every=50  # Save frequency
              )


I0000 00:00:1744210411.731111 14212531 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 2799.94it/s]

dataset has 4480 tokens
Training...


[10 | 149.36] loss=1.77 avg=1.77
[20 | 293.44] loss=0.88 avg=1.32
======== SAMPLE 1 ========
 its have all been hacked.
I was just talking in English about that.
It makes me want to eat your face.
Oh.
I'm sorry to bother you.
Oh.
Really.
What's wrong, Nick?
You know what I mean? I-- I have no choice. I-- I don't have a home. I don't have money. I can't afford a home. And I have no way to leave it. I have no place to go.
Okay.
What are you doing?
What-- what was that?
You were trying to tell me everything little Nick-- you know me really well. I mean, I'm one of the most intelligent people I know. I live next to you on a hill. I love you.
     (whispering) Oh. Shit.
Idk what you mean. I was trying to tell you everything because I just got an email. It's me, it's its - it's so embarrassing.
I know what you mean. I live next to you on a mountain. I'm in a river in California and I cant get anywhere - I have nowhere to live. I cant send my kids to see - I have nowhere to send them. I cant 

# Seeing What Happens!

Finally, with our trained model, I will show some outputs of text based on user-defined prefixes. In this prefix, I make it so that Samantha is complimenting the user 

In [12]:
gpt.generate(sess,
              model_name='124M',
              prefix="I love spending time with you",
              length=40,
              temperature=0.8,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

I love spending time with you. 
Extra time would be good for us. 
Will it be okay? 
Short of everything being exactly as it is? 
    (concerned) What are
I love spending time with you. 
Extra time would be good for us. 
Plus, we can have fun together. 
I think it would be good for us. 
Good for us. 

I love spending time with you. 
Lots of fun. 
But sincerely,
>>>>>>>>>>>>>>>>
>>>>>>>>  (whispering) What's wrong, Theodore? 
Is this something that youre not used
I love spending time with you. I miss you so much it hurts my whole body - give it a go!
How was it?
(unconvincing)
I dont know. (beat)
What are
I love spending time with you. 
Oh. Huh. Are those things usually done in person?
     (feeling off, but trying to be positive) Oh, brilliant. I really wanted to talk
